# Existance and Uniqueness of Solution

_Some python code to plot the functions_

In [1]:
import numpy as np
from ipywidgets import interact
import matplotlib.pyplot as plt


def plot_function(func, x1_range=(-5, 5), x2_range=(-5, 5)):
    x1 = np.linspace(min(x1_range), max(x1_range), 100)
    x2 = np.linspace(min(x2_range), max(x2_range), 100)
    X1, X2 = np.meshgrid(x1, x2)

    X1dot, X2dot = func(X1, X2)

    def update_plot(x1_start=1, x2_start=1):
        fig, ax = plt.subplots()
        ax.streamplot(X1, X2, X1dot, X2dot, linewidth=1)
        ax.plot(x1_start, x2_start, "ro", label="$x(0)$")
        start_points = np.array([[x1_start], [x2_start]]).T
        ax.streamplot(X1, X2, X1dot, X2dot, linewidth=2, start_points=start_points)
        ax.set_xlabel("$x_1$")
        ax.set_ylabel("$x_2$")
        ax.legend()
        ax.grid()
        plt.show()

    interact(
        update_plot,
        x1_start=(min(x1_range), max(x1_range),0.1),
        x2_start=(min(x2_range), max(x2_range),0.1),
    )

## Example 1

$$
\begin{align*}
\dot{x}_1 &= x_1 x_2 \\
\dot{x}_2 &= -x_2^2 - \sin(x_2) 
\end{align*}
$$

In [2]:
def func(x1, x2):
    return x1 * x2, -x1**2 - np.sin(x2)

plot_function(func=func, x1_range=(-10, 10), x2_range=(-10, 10))

interactive(children=(FloatSlider(value=1.0, description='x1_start', max=10.0, min=-10.0), FloatSlider(value=1…

# Example 2
$$
\begin{align*}
x \in \mathbb{R}^2, \quad \dot{x} &= Ax, \quad A = \begin{bmatrix} 1 & 2 \\ 3 & 4 \end{bmatrix}
\end{align*}
$$

In [3]:
def func(x1, x2):
    X1_flat = x1.flatten()
    X2_flat = x2.flatten()
    X = np.vstack((X1_flat, X2_flat))

    A = np.array([
        [1, 2],
        [3, 4]]
    )

    Xdot = A @ X

    X1dot = Xdot[0].reshape(x1.shape)
    X2dot = Xdot[1].reshape(x2.shape)
    return X1dot, X2dot

plot_function(func=func, x1_range=(-10, 10), x2_range=(-10, 10))

interactive(children=(FloatSlider(value=1.0, description='x1_start', max=10.0, min=-10.0), FloatSlider(value=1…

# Example 3
$$
\begin{align*}
x \in \mathbb{R}^2, \quad \dot{x} &= Ax, \quad A = \begin{bmatrix} 0 & 0 \\ 0 & -1 \end{bmatrix}
\end{align*}
$$

In [4]:
def func(x1, x2):
    X1_flat = x1.flatten()
    X2_flat = x2.flatten()
    X = np.vstack((X1_flat, X2_flat))

    A = np.array([
        [0, 0],
        [0, -1]]
    )

    Xdot = A @ X

    X1dot = Xdot[0].reshape(x1.shape)
    X2dot = Xdot[1].reshape(x2.shape)
    return X1dot, X2dot

plot_function(func=func, x1_range=(-10, 10), x2_range=(-10, 10))

interactive(children=(FloatSlider(value=1.0, description='x1_start', max=10.0, min=-10.0), FloatSlider(value=1…

# Example 4

$$
\begin{align*}
\dot{x} = \text{sign}(x) |x|^{1/3}
\end{align*}
$$

In [5]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
from scipy.integrate import solve_ivp

# 1. Define the function f(x)
def func1(x):
    # Handle x = 0 carefully if you want; np.sign(0) = 0, so it returns 0 anyway.
    return np.sign(x) * np.abs(x)**(1/3)

# 2. ODE function suitable for solve_ivp (time is first argument, state second)
def ode_func1(t, x):
    return func1(x)

# 3. A helper to plot the phase "portrait" (actually, for 1D, it's just f(x) vs x).
def plot_phase_line1(ax):
    x_vals = np.linspace(-4, 4, 1000)
    f_vals = [func1(x) for x in x_vals]
    ax.axhline(0, color='black', linewidth=0.8)
    ax.plot(x_vals, f_vals, label=r'$\dot{x} = f(x)$')
    ax.set_xlabel("x")
    ax.set_ylabel("dx/dt")
    ax.set_title("1D Phase Plot")
    ax.legend()
    # Mark equilibria (where f(x)=0) - obviously x=0 is an equilibrium
    ax.scatter(0, 0, color='red', zorder=5)
    ax.text(0.05, 0.05, "Equilibrium at x=0", color='red')

# 4. Interactive function to show the solution for a chosen initial condition
@interact(x0=FloatSlider(min=-2, max=2, step=0.01, value=0, description='Initial x0'))
def plot_trajectory1(x0):
    # We'll integrate from t=0 to t=5 (or more if you like)
    t_span = (0, 10)
    sol = solve_ivp(ode_func1, t_span, [x0], max_step=0.001, dense_output=True)
    t_dense = np.linspace(t_span[0], t_span[1], 300)
    x_dense = sol.sol(t_dense)[0]

    fig, ax = plt.subplots(1, 2, figsize=(12, 4))

    # Left plot: time evolution x(t)
    ax[0].plot(t_dense, x_dense, 'b')
    ax[0].set_xlabel('t')
    ax[0].set_ylabel('x(t)')
    ax[0].set_title(f'Trajectory starting at x0={x0:.2f}')
    ax[0].grid(True)

    # Right plot: phase line
    plot_phase_line1(ax[1])
    # Overplot the path in the (x, dx/dt) plane
    ax[1].plot(x_dense, func1(x_dense), 'r', label='Solution trajectory')
    ax[1].legend()

    plt.tight_layout()
    plt.show()


interactive(children=(FloatSlider(value=0.0, description='Initial x0', max=2.0, min=-2.0, step=0.01), Output()…

# Example 5

$$
\begin{align*}
\dot{x} = -x^2 
\end{align*}
$$

In [6]:
def func2(x):
    return -x**2

def ode_func2(t, x):
    return func2(x)

def plot_phase_line2(ax):
    x_vals = np.linspace(-4, 4, 1000)
    f_vals = [func2(x) for x in x_vals]
    ax.axhline(0, color='black', linewidth=0.8)
    ax.plot(x_vals, f_vals, label=r'$\dot{x} = f(x)$')
    ax.set_xlabel("x")
    ax.set_ylabel("dx/dt")
    ax.set_title("1D Phase Plot")
    ax.legend()
    # Mark equilibria (where f(x)=0) - obviously x=0 is an equilibrium
    ax.scatter(0, 0, color='red', zorder=5)
    ax.text(0.05, 0.05, "Equilibrium at x=0", color='red')

@interact(x0=FloatSlider(min=-2, max=2, step=0.01, value=0, description='Initial x0'))
def plot_trajectory2(x0):
    t_span = (0, 10)
    sol = solve_ivp(ode_func2, t_span, [x0], max_step=0.001, dense_output=True)
    t_dense = np.linspace(t_span[0], t_span[1], 300)
    x_dense = sol.sol(t_dense)[0]

    fig, ax = plt.subplots(1, 2, figsize=(12, 4))

    # Left plot: time evolution x(t)
    ax[0].plot(t_dense, x_dense, 'b')
    ax[0].set_xlabel('t')
    ax[0].set_ylabel('x(t)')
    ax[0].set_title(f'Trajectory starting at x0={x0:.2f}')
    ax[0].grid(True)

    # Right plot: phase line
    plot_phase_line2(ax[1])
    # Overplot the path in the (x, dx/dt) plane
    ax[1].plot(x_dense, func2(x_dense), 'r', label='Solution trajectory')
    ax[1].legend()

    plt.tight_layout()
    plt.show()


interactive(children=(FloatSlider(value=0.0, description='Initial x0', max=2.0, min=-2.0, step=0.01), Output()…

In [7]:
def func3(x):
    return -np.tanh(x)

def ode_func3(t, x):
    return func3(x)

def plot_phase_line3(ax):
    x_vals = np.linspace(-4, 4, 1000)
    f_vals = [func3(x) for x in x_vals]
    ax.axhline(0, color='black', linewidth=0.8)
    ax.plot(x_vals, f_vals, label=r'$\dot{x} = f(x)$')
    ax.set_xlabel("x")
    ax.set_ylabel("dx/dt")
    ax.set_title("1D Phase Plot")
    ax.legend()
    # Mark equilibria (where f(x)=0) - obviously x=0 is an equilibrium
    ax.scatter(0, 0, color='red', zorder=5)
    ax.text(0.05, 0.05, "Equilibrium at x=0", color='red')

# 4. Interactive function to show the solution for a chosen initial condition
@interact(x0=FloatSlider(min=-2, max=2, step=0.01, value=0, description='Initial x0'))
def plot_trajectory3(x0):
    # We'll integrate from t=0 to t=5 (or more if you like)
    t_span = (0, 10)
    sol = solve_ivp(ode_func3, t_span, [x0], max_step=0.001, dense_output=True)
    t_dense = np.linspace(t_span[0], t_span[1], 300)
    x_dense = sol.sol(t_dense)[0]

    fig, ax = plt.subplots(1, 2, figsize=(12, 4))

    # Left plot: time evolution x(t)
    ax[0].plot(t_dense, x_dense, 'b')
    ax[0].set_xlabel('t')
    ax[0].set_ylabel('x(t)')
    ax[0].set_title(f'Trajectory starting at x0={x0:.2f}')
    ax[0].grid(True)

    # Right plot: phase line
    plot_phase_line3(ax[1])
    # Overplot the path in the (x, dx/dt) plane
    ax[1].plot(x_dense, func3(x_dense), 'r', label='Solution trajectory')
    ax[1].legend()

    plt.tight_layout()
    plt.show()


interactive(children=(FloatSlider(value=0.0, description='Initial x0', max=2.0, min=-2.0, step=0.01), Output()…